#Fitting Binarized Neural Networks

In [ ]:
! pip install python-sat

In [ ]:
from pysat.formula import WCNF
from pysat.examples.rc2 import RC2
import itertools
import numpy as np
import operator
import time
from sklearn.metrics import accuracy_score
np.random.seed(42)

In [ ]:
time1=time.time()

In [ ]:
N = 5
P = 3
M = 3

In [ ]:
Nh = N//2+1
Ph = P//2+1

In [ ]:
Ncombinations = list(itertools.product([0, 1], repeat=N))

In [ ]:
Ncombinations

[(0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0),
 (0, 0, 0, 1, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 1),
 (0, 0, 1, 1, 0),
 (0, 0, 1, 1, 1),
 (0, 1, 0, 0, 0),
 (0, 1, 0, 0, 1),
 (0, 1, 0, 1, 0),
 (0, 1, 0, 1, 1),
 (0, 1, 1, 0, 0),
 (0, 1, 1, 0, 1),
 (0, 1, 1, 1, 0),
 (0, 1, 1, 1, 1),
 (1, 0, 0, 0, 0),
 (1, 0, 0, 0, 1),
 (1, 0, 0, 1, 0),
 (1, 0, 0, 1, 1),
 (1, 0, 1, 0, 0),
 (1, 0, 1, 0, 1),
 (1, 0, 1, 1, 0),
 (1, 0, 1, 1, 1),
 (1, 1, 0, 0, 0),
 (1, 1, 0, 0, 1),
 (1, 1, 0, 1, 0),
 (1, 1, 0, 1, 1),
 (1, 1, 1, 0, 0),
 (1, 1, 1, 0, 1),
 (1, 1, 1, 1, 0),
 (1, 1, 1, 1, 1)]

In [ ]:
np.random.shuffle(Ncombinations)

In [ ]:
train = Ncombinations[:int(len(Ncombinations)*0.65)]
test = Ncombinations[int(len(Ncombinations)*0.65):]

#fun with N=3,M=3
# def true_funct(x):
#   m=[]
#   #funct 1
#   if ((x[0] or x[1]) and (x[2])):
#     m.append(1)
#   else:
#     m.append(0)
#   #funct 2
#   if (x[0] or (x[1] and x[2])):
#     m.append(1)
#   else:
#     m.append(0)
#   #funct 3
#   if (not x[1]):
#     m.append(1)
#   else:
#     m.append(0)
#  return m

#fun with N=4,M=3
# def true_funct(x):
#   m=[]
#   #funct 1
#   if (((x[0] or x[1]) and (x[2] or x[3])) or (not x[0] and not x[2])):
#     m.append(1)
#   else:
#     m.append(0)
#   #funct 2
#   if ((x[0] and x[1] and x[2]) or (x[3])):
#     m.append(1)
#   else:
#     m.append(0)
#   #funct 3
#   if (not x[1] and not x[3]):
#     m.append(1)
#   else:
#     m.append(0)
#  return m

#fun with N=5,M=3
def true_funct(x):
  m=[]
  #funct 1
  if (((x[0] or x[1]) and (x[2] or x[3])) or (x[4] and (not x[0] and not x[2]))):
    m.append(1)
  else:
    m.append(0)
  #funct 2
  if ((x[0] and x[1] and x[2]) or (x[3] and x[4])):
    m.append(1)
  else:
    m.append(0)
  #funct 3
  if (not x[1] and not x[3] and not x[4]):
    m.append(1)
  else:
    m.append(0)
  return m

In [ ]:
y_train = [
    true_funct(x) for x in train
]

y_test = [
    true_funct(x) for x in test
]

In [ ]:
y_train

[[1, 1, 0],
 [1, 1, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [1, 1, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [1, 0, 0],
 [1, 1, 0],
 [1, 1, 0],
 [1, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [1, 1, 0]]

In [ ]:
def SATindex_x_i_n(i, n):
    return i * (len(train[0])) + n + 1
# get the index representing the i-th weight
def SATindex_w1_n_p(n, p):
  return len(train)*len(train[0])+(n*P)+p+1
def SATindex_w2_p_m(p,m):
  return len(train)*len(train[0])+(N*P)+(p*M)+m+1

In [ ]:
wcnf = WCNF()

In [ ]:
for i in range(len(train)):
    for j in range(len(train[0])):
        if train[i][j] > 0:
            wcnf.append([SATindex_x_i_n(i,j)])
        else:
            wcnf.append([-SATindex_x_i_n(i,j)])

In [ ]:
print(wcnf.hard)

[[1], [2], [3], [-4], [5], [-6], [7], [8], [9], [10], [11], [12], [-13], [-14], [-15], [16], [-17], [-18], [-19], [20], [-21], [22], [-23], [-24], [-25], [-26], [27], [-28], [-29], [30], [31], [32], [33], [34], [-35], [36], [37], [-38], [-39], [40], [-41], [42], [43], [-44], [-45], [-46], [-47], [-48], [-49], [-50], [-51], [-52], [53], [-54], [-55], [56], [-57], [-58], [-59], [-60], [-61], [-62], [63], [-64], [65], [-66], [67], [68], [-69], [70], [-71], [72], [-73], [74], [75], [76], [-77], [78], [79], [80], [-81], [-82], [-83], [-84], [85], [-86], [-87], [-88], [89], [-90], [91], [92], [-93], [94], [-95], [-96], [-97], [-98], [99], [100]]


In [ ]:
Ncomb = list(itertools.combinations(range(N), Nh))
Ncombneg = list(itertools.combinations(range(N), N-Nh+1))
Pcomb = list(itertools.combinations(range(P), Ph))
Pcombneg = list(itertools.combinations(range(P), P-Ph+1))
multNcomb = list(itertools.product([-1, 1], repeat=Nh))
multNcombneg = list(itertools.product([-1, 1], repeat=N-Nh+1))
multPcomb = list(itertools.product([-1, 1], repeat=Ph))
multPcombneg = list(itertools.product([-1, 1], repeat=P-Ph+1))

In [ ]:
def combine(l):
  tmpa = l[0]
  for lnum in range(1,len(l)):
    tmpb = np.asarray(list(itertools.product(tmpa, l[lnum])),dtype=object).tolist()
    tmpa = tmpb
    for q in range(len(tmpa)):
      for item in tmpa[q][1]:
        tmpa[q][0].append(item)
      tmpc=tmpa[q][0]
      tmpa[q]=tmpc
  return tmpa

In [ ]:
start = time.time()

for x_num in range(len(y_train)):
  for m in range(len(y_train[x_num])):
    if y_train[x_num][m]==1:
      print("positive, y: "+str(x_num+1)+", "+str(m+1))
      for i in Pcomb:
        tmp2 = [[] for z in itertools.repeat(None, len(i))]
        for p_num in range(len(i)):
          for k in multNcomb:
            for j in Ncomb:
              tmp=[]
              tmp.append(SATindex_w2_p_m(i[p_num],m))
              for num_n in range(len(j)):
                tmp.append(k[num_n]*SATindex_w1_n_p(j[num_n],i[p_num]))
                tmp.append(k[num_n]*SATindex_x_i_n(x_num,j[num_n]))
              tmp2[p_num].append(tmp)
          for k in multNcombneg:
            for j in Ncombneg:
              tmp=[]
              tmp.append(-SATindex_w2_p_m(i[p_num],m))
              for num_n in range(len(j)):
                tmp.append(k[num_n]*SATindex_w1_n_p(j[num_n],i[p_num]))
                tmp.append(-k[num_n]*SATindex_x_i_n(x_num,j[num_n]))
              tmp2[p_num].append(tmp)
        tmp3 = combine(tmp2)
        for lst in tmp3:
          wcnf.append(lst, weight=1)
    else:
      print("negative, y: "+str(x_num+1)+", "+str(m+1))
      for i in Pcombneg:
        tmp2 = [[] for z in itertools.repeat(None, len(i))]
        for p_num in range(len(i)):
          for k in multNcomb:
            for j in Ncomb:
              tmp=[]
              tmp.append(-SATindex_w2_p_m(i[p_num],m))
              for num_n in range(len(j)):
                tmp.append(k[num_n]*SATindex_w1_n_p(j[num_n],i[p_num]))
                tmp.append(k[num_n]*SATindex_x_i_n(x_num,j[num_n]))
              tmp2[p_num].append(tmp)
          for k in multNcombneg:
            for j in Ncombneg:
              tmp=[]
              tmp.append(SATindex_w2_p_m(i[p_num],m))
              for num_n in range(len(j)):
                tmp.append(k[num_n]*SATindex_w1_n_p(j[num_n],i[p_num]))
                tmp.append(-k[num_n]*SATindex_x_i_n(x_num,j[num_n]))
              tmp2[p_num].append(tmp)
        tmp3 = combine(tmp2)
        for lst in tmp3:
          wcnf.append(lst, weight=1)

end = time.time()
print(end - start)

positive, y: 1, 1
positive, y: 1, 2
negative, y: 1, 3
positive, y: 2, 1
positive, y: 2, 2
negative, y: 2, 3
negative, y: 3, 1
negative, y: 3, 2
negative, y: 3, 3
negative, y: 4, 1
negative, y: 4, 2
negative, y: 4, 3
negative, y: 5, 1
negative, y: 5, 2
negative, y: 5, 3
positive, y: 6, 1
negative, y: 6, 2
negative, y: 6, 3
positive, y: 7, 1
positive, y: 7, 2
negative, y: 7, 3
negative, y: 8, 1
negative, y: 8, 2
negative, y: 8, 3
positive, y: 9, 1
negative, y: 9, 2
negative, y: 9, 3
negative, y: 10, 1
negative, y: 10, 2
positive, y: 10, 3
negative, y: 11, 1
negative, y: 11, 2
positive, y: 11, 3
negative, y: 12, 1
negative, y: 12, 2
positive, y: 12, 3
negative, y: 13, 1
negative, y: 13, 2
negative, y: 13, 3
positive, y: 14, 1
negative, y: 14, 2
negative, y: 14, 3
positive, y: 15, 1
positive, y: 15, 2
negative, y: 15, 3
positive, y: 16, 1
positive, y: 16, 2
negative, y: 16, 3
positive, y: 17, 1
negative, y: 17, 2
negative, y: 17, 3
negative, y: 18, 1
negative, y: 18, 2
negative, y: 18, 3
p

In [ ]:
#doesn't work over 3,3,2 as N,P,M size
#print(wcnf.to_dimacs())

In [ ]:
start = time.time()
solver = RC2(wcnf)
end = time.time()
print(end - start)

23.841314554214478


In [ ]:
start = time.time()
model = solver.compute()
end = time.time()
print(end - start)

253.79344081878662


In [ ]:
model

[1,
 2,
 3,
 -4,
 5,
 -6,
 7,
 8,
 9,
 10,
 11,
 12,
 -13,
 -14,
 -15,
 16,
 -17,
 -18,
 -19,
 20,
 -21,
 22,
 -23,
 -24,
 -25,
 -26,
 27,
 -28,
 -29,
 30,
 31,
 32,
 33,
 34,
 -35,
 36,
 37,
 -38,
 -39,
 40,
 -41,
 42,
 43,
 -44,
 -45,
 -46,
 -47,
 -48,
 -49,
 -50,
 -51,
 -52,
 53,
 -54,
 -55,
 56,
 -57,
 -58,
 -59,
 -60,
 -61,
 -62,
 63,
 -64,
 65,
 -66,
 67,
 68,
 -69,
 70,
 -71,
 72,
 -73,
 74,
 75,
 76,
 -77,
 78,
 79,
 80,
 -81,
 -82,
 -83,
 -84,
 85,
 -86,
 -87,
 -88,
 89,
 -90,
 91,
 92,
 -93,
 94,
 -95,
 -96,
 -97,
 -98,
 99,
 100,
 -101,
 -102,
 103,
 104,
 -105,
 106,
 107,
 108,
 109,
 110,
 -111,
 112,
 113,
 -114,
 115,
 116,
 117,
 -118,
 -119,
 -120,
 121,
 -122,
 -123,
 124]

In [ ]:
w1=np.sign(model[len(train)*len(train[0]):len(train)*len(train[0])+(N*P)]).reshape(N,P)

In [ ]:
w1

array([[-1, -1,  1],
       [ 1, -1,  1],
       [ 1,  1,  1],
       [ 1, -1,  1],
       [ 1, -1,  1]])

In [ ]:
w2=np.sign(model[len(train)*len(train[0])+(N*P):]).reshape(P,M)

In [ ]:
w2

array([[ 1,  1, -1],
       [-1, -1,  1],
       [-1, -1,  1]])

In [ ]:
pred_train = np.sign(np.matmul(np.sign(np.matmul(train,w1)-0.5),w2)-0.5)
pred_test = np.sign(np.matmul(np.sign(np.matmul(test,w1)-0.5),w2)-0.5)

In [ ]:
print(f"Training acc: {accuracy_score(list(itertools.chain(*np.sign(np.array(y_train)-0.5))), list(itertools.chain(*pred_train)))}")
print(f"Test acc: {accuracy_score(list(itertools.chain(*np.sign(np.array(y_test)-0.5))), list(itertools.chain(*pred_test)))}")

Training acc: 0.7
Test acc: 0.7222222222222222


In [ ]:
print(time.time() - time1)

319.70762038230896
